In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import cigvis
from cigvis import colormap
import plotly
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
root = '/Users/lijintao/Downloads/data/F3/'
seisp = root + 'seis.dat'
saltp = root + 'salt.dat'
hz2p = root + 'hz.dat'
unc1p = root + 'unc1.dat'
unc2p = root + 'unc2.dat'
ni, nx, nt = 591, 951, 362
shape = (ni, nx, nt)



In [ ]:
def load_well_logs(logsp):
    nlog = 4
    npoints = 2121
    x = [259, 619, 339, 141]
    y = [33, 545, 704, 84]
    z = np.arange(0, 0.2 * npoints, 0.2)
    v = np.fromfile(logsp, np.float32).reshape(nlog, npoints)
    v = 0.5 * np.log(v)
    nodes = []
    for i in range(nlog):
        points = np.c_[np.ones(npoints) * x[i], np.ones(npoints) * y[i], z, v[i]]
        nodes += cigvis.create_Line_logs(points)
    return nodes

In [ ]:
# seismic
seis = np.memmap(seisp, np.float32, 'c', shape=shape)
# overlay
inter = np.memmap(root + 'overlay.dat', np.float32, 'c', shape=shape)

salt = np.memmap(saltp, np.float32, 'c', shape=shape)
hz2 = np.fromfile(hz2p, np.float32).reshape(ni, nx)
unc = np.fromfile(root + 'unc.dat', np.float32).reshape(shape)
unc2 = np.fromfile(unc2p, np.float32).reshape(ni, nx).astype(np.float32)

In [ ]:
fg_cmap = colormap.set_alpha('jet', 0.6, False)
fg_clim = [inter.max() * 0.15, inter.max() * 0.5]
nodes = []
nodes += cigvis.create_overlay(seis, 
                              inter,
                              pos=[ni - 2, 25, nt - 2], 
                              bg_cmap='gray', 
                              bg_clim=[-2.0, 1.5], 
                              fg_cmap=fg_cmap,
                              fg_clim=fg_clim,)

nodes += cigvis.create_bodys(salt, 0.0, 0.0, color='cyan')
nodes += cigvis.create_surfaces([hz2], volume=seis, value_type='amp', 
                                cmap='gray', 
                                clim=[-2.0, 1.5])
nodes += cigvis.create_surfaces([unc2], volume=unc, value_type='amp')
nodes += load_well_logs(root+'logs.dat')


cigvis.plot3D(nodes, 
              aspectratio=dict(x=ni/nx,y=1,z=nt/nx*1.7), 
              aspectmode='manual',
              savequality=5)